In [2]:
#Importing configuration
import configparser
config = configparser.ConfigParser()
config.read("config.txt")

word2vec_path = config.get("configuration","word2vec_path")
stanford_corenlp_path = config.get("configuration","stanford_corenlp_path")


import pymysql
pymysql.install_as_MySQLdb()

# Connect
mydbhost = config.get("configuration","mydbhost")
mydbuser = config.get("configuration","mydbuser")
mydbpasswd = config.get("configuration","mydbpasswd")
mydbdb = config.get("configuration","mydbdb")

In [3]:
from stanfordcorenlp import StanfordCoreNLP
# Tree syntax of natural language: http://www.cs.cornell.edu/courses/cs474/2004fa/lec1.pdf

In [4]:
# Importing word2vec to find similarity and neighboring words
import gensim
from gensim.models import Word2Vec

model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True, limit=500000) 

In [5]:
# importing StandfordCoreNLP to tokenize, tag, and ner
nlp = StanfordCoreNLP(stanford_corenlp_path)

#sentence = "The hacker group Anonymous installed ransomware to steal cryptocurrency."
#sentence = "ISIL members launched DDOS attack on European Union."
sentence = "Russian hackers sent phishing emails to compromise NCSU's backup servers."

sentence_tokens = nlp.word_tokenize(sentence)
sentence_tags = nlp.pos_tag(sentence)
sentence_ner = nlp.ner(sentence)
#sentence_parse = nlp.parse(sentence)
#sentence_dependency = nlp.dependency_parse(sentence)

print('Sentence:', sentence)
# print('Tokenize:', nlp.word_tokenize(sentence))
print('Part of Speech:', nlp.pos_tag(sentence))
print('Named Entities:', nlp.ner(sentence))
#print('Constituency Parsing:', nlp.parse(sentence))
#print('Dependency Parsing:', nlp.dependency_parse(sentence))

to_replace_ners = []
to_replace_verbs = []
to_replace_verbphrases = []
to_replace_adjectives = []
to_replace_adjphrase = []
# prev_ner = ""
# current_entity = ""
# prev_entity_exists = 0

for (i, j) in sentence_ner:
#     print(i, j)
    if(j!='O'):
        to_replace_ners.append((i, j))
#         print('PREV NER', prev_ner, 'Current NER', j)
#         if(j==prev_ner): 
#             current_entity = current_entity + '_' + i
#             print(current_entity)
#             prev_ner = j
#             prev_entity_exists = 1
#         elif(prev_entity_exists): 
#             to_replace_ners.append((current_entity, prev_ner))
#             prev_entity_exists = 0
#             current_entity = i
#             prev_ner = j
#     elif(current_entity != ""):     
#         to_replace_ners.append((current_entity, prev_ner))
#         prev_ner = ""
#         current_entity = ""
    
        
verb_check = 0
adj_check = 0
        
for (i, j) in sentence_tags:
    if(verb_check == 1):
        verbphrase = verb + '_' + i
        to_replace_verbphrases.append((verbphrase, i))
        verb_check = 0
        #print(verbphrase)
    
    if(j=='VBD' or j=='VBZ' or j == 'VBP' or j == 'VBN'):
        #print(i, j)
        to_replace_verbs.append(i)
        verb_check = 1
        verb = i
    
    if(j == 'JJ'):
        to_replace_adjectives.append(i)
        adj = j
        adj_check = 1

# to_replace_entities = []
# prev_type = ""
# prev_entity = ""
# cur_entity = ""
# lastindex = -1
# for (entity, type) in to_replace_ners:
#     print(entity, type, prev_entity, prev_type, cur_entity)
    
#     if(type==prev_type):
#         to_replace_entities[lastindex] = (to_replace_ners[lastindex][0] + '_' + entity, type)
#     else:
#         to_replace_entities.append((cur_entity, prev_type))
#         cur_entity = entity
#         lastindex = lastindex + 1
    
#     prev_type = type
#     #prev_entity = entity

print(to_replace_ners)
# print(to_replace_entities)
print(to_replace_verbs)
print(to_replace_verbphrases)
print(to_replace_adjectives)
        
nlp.close()

Sentence: Russian hackers sent phishing emails to compromise NCSU's backup servers.
Part of Speech: [('Russian', 'JJ'), ('hackers', 'NNS'), ('sent', 'VBN'), ('phishing', 'VBG'), ('emails', 'NNS'), ('to', 'TO'), ('compromise', 'VB'), ('NCSU', 'NNP'), ("'s", 'POS'), ('backup', 'NN'), ('servers', 'NNS'), ('.', '.')]
Named Entities: [('Russian', 'NATIONALITY'), ('hackers', 'O'), ('sent', 'O'), ('phishing', 'O'), ('emails', 'O'), ('to', 'O'), ('compromise', 'O'), ('NCSU', 'ORGANIZATION'), ("'s", 'O'), ('backup', 'O'), ('servers', 'O'), ('.', 'O')]
[('Russian', 'NATIONALITY'), ('NCSU', 'ORGANIZATION')]
['sent']
[('sent_phishing', 'phishing')]
['Russian']


In [6]:
topk = 10
replacement_ners = []
replacement_verbs = []
replacement_verbphrases = []
replacement_nouns = []
replacement_adjectives = []

for (i, j) in to_replace_ners:
    similar_ners = model.most_similar([i, j.lower()], [], topk)
    replacement_ners.append((i, similar_ners))

print(replacement_ners)
    
for verb in to_replace_verbs:
    similar_verbs = model.most_similar(verb, [], topk)
    replacement_verbs.append((verb,similar_verbs))

print(replacement_verbs)

for (verbphrase, nn) in to_replace_verbphrases:
    try:
        similar_verbphrases = model.most_similar([verbphrase, nn], [], topk)
        replacement_verbphrases.append((verbphrase, similar_verbphrases))
    except KeyError as e:
        print(e)

print(replacement_verbphrases)

for (verbphrase, nn) in to_replace_verbphrases:
    try:
        similar_nouns = model.most_similar(nn, [], topk)
        replacement_nouns.append((nn, similar_nouns))
    except KeyError as e:
        print(e)

print(replacement_nouns)

for adjective in to_replace_adjectives: 
    similar_adjectives = model.most_similar(adjective, [], topk)
    replacement_adjectives.append((adjective, similar_adjectives))
    
print(replacement_adjectives)

[('Russian', [('Ukrainian', 0.6549570560455322), ('Kazakh', 0.6192217469215393), ('Latvian', 0.613041341304779), ('Bulgarian', 0.6038623452186584), ('Belarussian', 0.6027601957321167), ('Russians', 0.6012547016143799), ('Ukranian', 0.5897480249404907), ('Romanian', 0.5878135561943054), ('Uzbek', 0.5867130160331726), ('Belarusian', 0.5811043977737427)]), ('NCSU', [('UNC', 0.5804125070571899), ('UNC_CH', 0.5564162135124207), ('UNC_Charlotte', 0.5349904894828796), ('UNC_Chapel_Hill', 0.5265216827392578), ('UNCC', 0.5255768299102783), ('NCCU', 0.511374294757843), ('IPTAY', 0.505825400352478), ('NC', 0.49392879009246826), ('U.Va.', 0.49198469519615173), ('WSSU', 0.4856509566307068)])]
[('sent', [('sending', 0.7625813484191895), ('send', 0.736850917339325), ('sends', 0.645559549331665), ('forwarded', 0.618112325668335), ('mailed', 0.5840898752212524), ('despatched', 0.5625948309898376), ('Sending', 0.5511707663536072), ('dispatched', 0.5449301600456238), ('e_mailed', 0.5339891910552979), ('c

In [7]:
# Listing more attacks like 'phishing' based on W2V

model.most_similar(['phishing'], [], 50)

[('phishing_attacks', 0.8331868052482605),
 ('phishing_scams', 0.8290295600891113),
 ('Phishing', 0.8181643486022949),
 ('phishers', 0.7731481790542603),
 ('phish', 0.7567018270492554),
 ('phishing_emails', 0.7406015396118164),
 ('phishing_e_mails', 0.7268457412719727),
 ('malware', 0.7232787609100342),
 ('phishing_schemes', 0.7223339676856995),
 ('phishing_scam', 0.7034322023391724),
 ('phisher', 0.7024546265602112),
 ('vishing', 0.6804668307304382),
 ('Phishing_scams', 0.6792201399803162),
 ('cybercriminals', 0.6781081557273865),
 ('keylogging', 0.6731592416763306),
 ('antiphishing', 0.6637416481971741),
 ('Phishers', 0.6624651551246643),
 ('scareware', 0.6609221696853638),
 ('spyware', 0.6607507467269897),
 ('Zeus_Trojan', 0.6595767140388489),
 ('ransomware', 0.6550052165985107),
 ('cyber_criminals', 0.6540520787239075),
 ('spear_phishing', 0.653903603553772),
 ('trojan', 0.6471049785614014),
 ('spammers', 0.6431359052658081),
 ('Vishing', 0.6406604647636414),
 ('crimeware', 0.63811

In [13]:
# Get categories for NCSU from Wikipedia
# Need to hardcode NCSU as North_Carolina_State_University as there is no instance as NCSU in Wikipedia ontology

connection = pymysql.connect(host=mydbhost,
                             user=mydbuser,
                             passwd=mydbpasswd,
                             db=mydbdb)

replacement_institutes = []

try:
    with connection.cursor() as cursor:
        # Execute SQL select statement
        cursor.execute("SELECT instance, class FROM simple_types where instance like '%North_Carolina_State_University%'")
        # Commit your changes if writing
        # In this case, we are only reading data
        # db.commit()
        
        # Get the number of rows in the resultset
        numrows = cursor.rowcount
        
        # Get and display one row at a time
        for x in range(0, numrows):
            row = cursor.fetchone()
            print(row[0], "-->", row[1])
            replacement_institutes.append(('NCSU', row[1]))
# Close the connection
finally:
    # Close connection.
    connection.close()

<North_Carolina_State_University_College_of_Veterinary_Medicine> --> <wikicat_Veterinary_schools_in_the_United_States>
<North_Carolina_State_University_Insect_Museum> --> <wikicat_Natural_history_museums_in_North_Carolina>
<North_Carolina_State_University_Insect_Museum> --> <wordnet_museum_103800563>
<North_Carolina_State_University_reactor_program> --> <wordnet_reactor_104057846>
<North_Carolina_State_University> --> <wikicat_Educational_institutions_established_in_1887>
<North_Carolina_State_University> --> <wikicat_Universities_and_colleges_in_North_Carolina>
<Centennial_Campus_of_North_Carolina_State_University> --> <wikicat_Neighborhoods_in_Raleigh,_North_Carolina>
<North_Carolina_State_University_reactor_program> --> <wikicat_Nuclear_research_reactors>
<North_Carolina_State_University_Insect_Museum> --> <wikicat_Museums_in_Raleigh,_North_Carolina>
<North_Carolina_State_University_Insect_Museum> --> <wikicat_Research_museums_in_the_United_States>
<Centennial_Campus_of_North_Caroli

In [14]:
# Listing category based hypotheses 

category_hypotheses = []

for (institute, category) in replacement_institutes: 
    category_hypotheses.append(sentence.replace(institute, category))
    
print("Listing category based hypotheses")
print(category_hypotheses)

Listing category based hypotheses
["Russian hackers sent phishing emails to compromise <wikicat_Veterinary_schools_in_the_United_States>'s backup servers.", "Russian hackers sent phishing emails to compromise <wikicat_Natural_history_museums_in_North_Carolina>'s backup servers.", "Russian hackers sent phishing emails to compromise <wordnet_museum_103800563>'s backup servers.", "Russian hackers sent phishing emails to compromise <wordnet_reactor_104057846>'s backup servers.", "Russian hackers sent phishing emails to compromise <wikicat_Educational_institutions_established_in_1887>'s backup servers.", "Russian hackers sent phishing emails to compromise <wikicat_Universities_and_colleges_in_North_Carolina>'s backup servers.", "Russian hackers sent phishing emails to compromise <wikicat_Neighborhoods_in_Raleigh,_North_Carolina>'s backup servers.", "Russian hackers sent phishing emails to compromise <wikicat_Nuclear_research_reactors>'s backup servers.", "Russian hackers sent phishing email

In [130]:
# Universities with nuclear research reactor
connection = pymysql.connect(host=mydbhost,
                             user=mydbuser,
                             passwd=mydbpasswd,
                             db=mydbdb)

try:
    with connection.cursor() as cursor:
        # Execute SQL select statement
        cursor.execute("SELECT instance, class FROM simple_types where class like '%wikicat_Nuclear_research_reactors%' and instance like '%university%'")
        
        # Get the number of rows in the resultset
        numrows = cursor.rowcount
        
        # Get and display one row at a time
        for x in range(0, numrows):
            row = cursor.fetchone()
            print(row[0], "-->", row[1])
            
# Close the connection
finally:
    # Close connection.
    connection.close()

<Washington_State_University_Reactor> --> <wikicat_Nuclear_research_reactors>
<Oregon_State_University_Radiation_Center> --> <wikicat_Nuclear_research_reactors>
<North_Carolina_State_University_reactor_program> --> <wikicat_Nuclear_research_reactors>
<Purdue_University_Reactor_Number_One> --> <wikicat_Nuclear_research_reactors>
<University_of_Massachusetts_Lowell_Radiation_Laboratory> --> <wikicat_Nuclear_research_reactors>


In [131]:
# Universities museums in NC
connection = pymysql.connect(host=mydbhost,
                             user=mydbuser,
                             passwd=mydbpasswd,
                             db=mydbdb)

try:
    with connection.cursor() as cursor:
        # Execute SQL select statement
        cursor.execute("SELECT instance, class FROM simple_types where class like '%wikicat_University_museums_in_North_Carolina%'")
        
        # Get the number of rows in the resultset
        numrows = cursor.rowcount
        
        # Get and display one row at a time
        for x in range(0, numrows):
            row = cursor.fetchone()
            print(row[0], "-->", row[1])
            
# Close the connection
finally:
    # Close connection.
    connection.close()

<Weatherspoon_Art_Museum> --> <wikicat_University_museums_in_North_Carolina>
<Ackland_Art_Museum> --> <wikicat_University_museums_in_North_Carolina>
<Nasher_Museum_of_Art> --> <wikicat_University_museums_in_North_Carolina>
<North_Carolina_State_University_Insect_Museum> --> <wikicat_University_museums_in_North_Carolina>


In [133]:
model.most_similar(['NCSU', 'China'], ['USA'], 50)

[('UNC_CH', 0.506248414516449),
 ('UNC_Chapel_Hill', 0.5013554692268372),
 ('Tsinghua', 0.4970877766609192),
 ('Anhui', 0.48243093490600586),
 ('Liaoning', 0.48201996088027954),
 ('Fudan', 0.4745294451713562),
 ('Zhejiang', 0.4721640944480896),
 ('yuan', 0.4709556996822357),
 ('NCCU', 0.47059300541877747),
 ('Tsinghua_University', 0.4681386947631836),
 ('Guangdong', 0.4578723609447479),
 ('Guizhou', 0.45680493116378784),
 ('Hubei', 0.4539087116718292),
 ('Yuan', 0.45208263397216797),
 ('Anhui_province', 0.45119917392730713),
 ('Yunnan', 0.4510120153427124),
 ('Shaanxi', 0.4507513642311096),
 ('China_Hunan_Province', 0.4505188465118408),
 ('UNC_Charlotte', 0.4503973424434662),
 ('trillion_yuan', 0.4488682150840759),
 ('Sichuan', 0.4478117525577545),
 ('Peking_University', 0.4472169578075409),
 ('Liaoning_province', 0.4455345571041107),
 ('Guangxi_Zhuang_Autonomous_Region', 0.4436618685722351),
 ('Yunnan_province', 0.4407528042793274),
 ('Jilin_Province', 0.4382138252258301),
 ('Chinese'